In [3]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

import numpy as np
import sys
sys.path.append("..")
import pprint

from collections import defaultdict

from reference.lib.envs.blackjack import BlackjackEnv
from reference.lib import plotting

matplotlib.style.use('ggplot')

In [4]:
env = BlackjackEnv()

In [98]:
def mc_prediction(policy, env, num_episodes, discount_factor=1.0):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        policy: A function that maps an observation to action probabilities.
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of sum and count of returns for each state to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum   = defaultdict(float)
    returns_count = defaultdict(float)

    # The final value function
    V = defaultdict(float)
    
    # start the loop
    for _ in range(num_episodes):
        
        # reset the state 
        observation = env.reset()
        action_to_take = sample_policy(observation)
        next_observation,reward,game_over,_ = env.step(action_to_take)
        
        if observation in returns_count: returns_count[observation] = returns_count[observation] + 1
        else:                            returns_count[observation] = 1
        
        
        current_state_to_estimate = observation[0]

        if current_state_to_estimate in returns_count:
            returns_count[current_state_to_estimate] = returns_count[current_state_to_estimate] + 1
        else:
            returns_count[current_state_to_estimate] = 1

        action_to_take = sample_policy(observation)
        observation,reward,game_over,_ = env.step(action_to_take)

        while not game_over: 
            action_to_take = sample_policy(observation)
            observation,reward,game_over,_ = env.step(action_to_take)

        if current_state_to_estimate in returns_count:
            returns_sum[current_state_to_estimate] = returns_sum[current_state_to_estimate] + reward
        else:
            returns_sum[current_state_to_estimate] = reward

    for x in returns_sum.keys():
        V[x] = returns_sum[x]/returns_count[x]

    return V    

In [99]:
def sample_policy(observation):
    """
    A policy that sticks if the player score is > 20 and hits otherwise.
    """
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

In [100]:
V_10k = mc_prediction(sample_policy, env, num_episodes=10000)
plotting.plot_value_function(V_10k, title="10,000 Steps")

V_500k = mc_prediction(sample_policy, env, num_episodes=500000)
plotting.plot_value_function(V_500k, title="500,000 Steps")

IndexError: invalid index to scalar variable.

In [101]:
V_10k

defaultdict(float,
            {21: 0.8735294117647059,
             17: -0.657051282051282,
             13: -0.582089552238806,
             12: -0.5525862068965517,
             18: -0.6181015452538632,
             16: -0.670490093847758,
             14: -0.5636025998142989,
             20: 0.6028636021100227,
             15: -0.6030977734753146,
             19: -0.6819923371647509})

In [89]:
returns_sum   = defaultdict(float)
returns_count = defaultdict(float)

# The final value function
V = defaultdict(float)
player,dealer,game_over = env.reset()

for _ in range(10):
    observation = env.reset()
    current_state_to_estimate =observation[0]

    if current_state_to_estimate in returns_count:
        returns_count[current_state_to_estimate] = returns_count[current_state_to_estimate] + 1
    else:
        returns_count[current_state_to_estimate] = 1

    action_to_take = sample_policy(observation)
    observation,reward,game_over,_ = env.step(action_to_take)

    while not game_over: 
        action_to_take = sample_policy(observation)
        observation,reward,game_over,_ = env.step(action_to_take)

    if current_state_to_estimate in returns_count:
        returns_sum[current_state_to_estimate] = returns_sum[current_state_to_estimate] + reward
    else:
        returns_sum[current_state_to_estimate] = reward

for x in returns_sum.keys():
    V[x] = returns_sum[x]/returns_count[x]

In [90]:
print(returns_sum.keys())
print(returns_count.keys())

dict_keys([14, 12, 20, 18, 19, 16, 13, 17])
dict_keys([14, 12, 20, 18, 19, 16, 13, 17])


In [95]:
V

defaultdict(float,
            {14: -1.0,
             12: -1.0,
             20: 1.0,
             18: -1.0,
             19: -1.0,
             16: 1.0,
             13: -1.0,
             17: -1.0})

In [52]:
env.step(1)

((117, 5, False), -1, True, {})

In [10]:
for x in dir(env):
    temp = getattr(env,x)
    print(x,' : ', temp)

__class__  :  <class 'reference.lib.envs.blackjack.BlackjackEnv'>
__delattr__  :  <method-wrapper '__delattr__' of BlackjackEnv object at 0x000001B3C01B4E80>
__dict__  :  {'action_space': Discrete(2), 'observation_space': Tuple(Discrete(32), Discrete(11), Discrete(2)), 'np_random': <mtrand.RandomState object at 0x000001B3C2F1D0D8>, 'natural': False, 'dealer': [10, 2], 'player': [6, 1], 'nA': 2}
__dir__  :  <built-in method __dir__ of BlackjackEnv object at 0x000001B3C01B4E80>
__doc__  :  Simple blackjack environment
    Blackjack is a card game where the goal is to obtain cards that sum to as
    near as possible to 21 without going over.  They're playing against a fixed
    dealer.
    Face cards (Jack, Queen, King) have point value 10.
    Aces can either count as 11 or 1, and it's called 'usable' at 11.
    This game is placed with an infinite deck (or with replacement).
    The game starts with each (player and dealer) having one face up and one
    face down card.
    The player c